In [4]:
import os
import pandas as pd
from qiime2 import Visualization
import matplotlib.pyplot as plt
import numpy as np

import qiime2 as q2

%matplotlib inline

# location of this week's data and all the results produced by this notebook 
# - this should be a path relative to your working directory
Data = 'Data'

In [6]:
#getting data 
# Download the entire folder as a zip
! wget "https://polybox.ethz.ch/index.php/s/bbPNGZPpGwBHBct/download" -O polybox_files.zip

# Unzip into data_dir
! unzip polybox_files.zip -d Data

# Clean up the zip file
! rm polybox_files.zip

--2025-10-21 11:34:34--  https://polybox.ethz.ch/index.php/s/bbPNGZPpGwBHBct/download
Resolving polybox.ethz.ch (polybox.ethz.ch)... 129.132.71.243
Connecting to polybox.ethz.ch (polybox.ethz.ch)|129.132.71.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘polybox_files.zip’

polybox_files.zip       [ <=>                ]   2.29M  --.-KB/s    in 0.1s    

2025-10-21 11:34:34 (19.0 MB/s) - ‘polybox_files.zip’ saved [2397226]

Archive:  polybox_files.zip
   creating: Data/ITS/
 extracting: Data/ITS/20250913_metadata_ITS.tsv  
 extracting: Data/ITS/20250914_metadata_personal_environmental_sensory_details.tsv  
 extracting: Data/ITS/filtered-dada1-rep-seqs-highschool.qza  
 extracting: Data/ITS/modified-dada1-table-highschool.qza  


In [2]:
df_meta_ITS = pd.read_csv('Data/20250913_metadata_ITS.tsv', sep='\t')
df_meta_ITS.head()

,sample ID,label_barcode1,label_barcode2,created_date,amplicon,unique_id,sample_id,person-id,plate,plate_position,amplicon_conc,DNA_extraction_plate,project,project_2,sample_type,day,hand
0,366291_386-LP4-ITS-0386,GGAAGTGAAGTT,ATTTGCTTTGCC,16.10.2024 14:59,ITS,LP4-ITS-0386,1820-d21-1,1820.0,P5,B01,6.7035,DNA59,highschool,highschool,sourdough,21.0,NaN
1,366291_387-LP4-ITS-0387,GATCGGGTCGGA,GTGACCCTGTCA,16.10.2024 14:59,ITS,LP4-ITS-0387,1725-d7-1,1725.0,P5,C01,7.8515,DNA59,highschool,highschool,sourdough,7.0,NaN
2,366291_388-LP4-ITS-0388,GTGACCTGATGT,GAGAATGGAAAG,16.10.2024 14:59,ITS,LP4-ITS-0388,1806-d14-1,1806.0,P5,D01,10.1119,DNA59,highschool,highschool,sourdough,14.0,NaN
3,366291_389-LP4-ITS-0389,GCCTACCGCCTA,CAGTAGCGATAT,16.10.2024 14:59,ITS,LP4-ITS-0389,1723-d21-1,1723.0,P5,E01,5.0523,DNA59,highschool,highschool,sourdough,21.0,NaN
4,366291_390-LP4-ITS-0390,ATACGGGCGCAT,GCCAAGGATAGG,16.10.2024 14:59,ITS,LP4-ITS-0390,1706-d7-1,1706.0,P5,F01,6.1151,DNA59,highschool,highschool,sourdough,7.0,NaN


In [6]:
df_meta_details = pd.read_csv('Data/20250914_metadata_personal_environmental_sensory_details.tsv', sep='\t')
df_meta_details.head()

,person-id,start_time,completion_time,background,ethical_agreement,sd_bake_experience,sd_bake_last_time,yeast_bake_experience,yeast_bake_last_time,sd_stor_loc,...,animal_score_D28,chemical_score_D28,body_odour_score_D28,fruity_score_D28,maillard_score_D28,sour_score_D28,ocean_score_D28,earthy_score_D28,fermented_dairy_score_D28,nutty_score_D28
0,1701,6.18.24 23:34:47,6.18.24 23:37:55,sterile,I agree.,no,NaN,yes,4.18.2024,kitchen counter,...,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1702,6.18.24 23:37:40,6.18.24 23:42:32,sterile,I agree.,no,NaN,yes,05.01.2023,cupboard,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1703,6.18.24 15:24:39,6.18.24 15:28:40,sterile,I agree.,no,NaN,yes,NaN,kitchen counter,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0
3,1704,6.24.24 21:14:24,6.24.24 21:16:48,sterile,I agree.,no,NaN,no,NaN,kitchen counter,...,0.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0
4,1705,6.24.24 21:41:15,6.24.24 21:46:29,sterile,I agree.,no,NaN,yes,2.19.2024,kitchen counter,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Rausnehmen:
- ethical agreement die nicht zustimmen rausfiltern, dann diese Zeile löschen

In [24]:
! qiime tools peek $'Data/filtered-dada1-rep-seqs-highschool.qza'

UUID:        f4cd20d6-49d4-405e-810f-e207abfeca8f
Type:        FeatureData[Sequence]
Data format: DNASequencesDirectoryFormat


In [3]:
!qiime feature-table tabulate-seqs \
  --i-data Data/filtered-dada1-rep-seqs-highschool.qza \
  --o-visualization Data/filtered-dada1-rep-seqs-highschool.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Usage: qiime feature-table tabulate-seqs [OPTIONS]

  Generate tabular view of feature identifier to sequence mapping, including
  links to BLAST each sequence against the NCBI nt database.

Inputs:
  --i-data ARTIFACT FeatureData[Sequence | AlignedSequence]
                         The feature sequences to be tabulated.     [required]
  --i-taxonomy ARTIFACTS... Collection[FeatureData[Taxonomy]]
                         The taxonomic classifications of the tabulated
                         features.                                  [optional]
Parameters:
  --m-metadata-file METADATA...
    (multiple            Any additional metadata for the tabulated features.


In [29]:
Visualization.load(f"{Data}/filtered-dada1-rep-seqs-highschool.qzv")

<visualization: Visualization uuid: 9de3b6ca-cd7c-429c-a8c1-26a6faa99830>

In [34]:
! qiime tools peek $'Data/modified-dada1-table-highschool.qza'

UUID:        830e5815-f7da-44ed-9a12-9ec71b92c448
Type:        FeatureTable[Frequency]
Data format: BIOMV210DirFmt


In [35]:
!qiime feature-table summarize \
  --i-table Data/modified-dada1-table-highschool.qza \
  --o-visualization Data/modified-dada1-table-highschool-summary.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Data/modified-dada1-table-highschool-summary.qzv


In [38]:
Visualization.load(f"{Data}/modified-dada1-table-highschool-summary.qzv")

<visualization: Visualization uuid: e63be93a-d0b2-4681-be53-96b093aff0d1>